In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Data preparation

In [ ]:
!unzip "/content/drive/MyDrive/001_projects/FSL/FSL_annotations/CIGNA_DENTAL_ADA/batch_1/ada_batch_1_augmented_yolov8.zip"

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define paths
root_folder = "/content/ada_batch_1_augmented_yolov8"
images_folder = os.path.join(root_folder, "images")
labels_folder = os.path.join(root_folder, "labels")

# Create train and val folders
root_train_folder = os.path.join("data", "images", "train")
root_val_folder = os.path.join("data", "images", "valid")
root_train_labels_folder = os.path.join("data", "labels", "train")
root_val_labels_folder = os.path.join("data", "labels", "valid")

os.makedirs(root_train_folder, exist_ok=True)
os.makedirs(root_val_folder, exist_ok=True)
os.makedirs(root_train_labels_folder, exist_ok=True)
os.makedirs(root_val_labels_folder, exist_ok=True)

# Get list of image files
image_files = [f for f in os.listdir(images_folder) if os.path.isfile(os.path.join(images_folder, f))]

# Split the dataset
train_images, val_images = train_test_split(image_files, test_size=0.01, random_state=42)

# Copy images to train folder
for image in train_images:
    shutil.copy(os.path.join(images_folder, image), os.path.join(root_train_folder, image))

# Copy labels to train folder
for image in train_images:
    label_file = os.path.splitext(image)[0] + ".txt"
    shutil.copy(os.path.join(labels_folder, label_file), os.path.join(root_train_labels_folder, label_file))

# Copy images to val folder
for image in val_images:
    shutil.copy(os.path.join(images_folder, image), os.path.join(root_val_folder, image))

# Copy labels to val folder
for image in val_images:
    label_file = os.path.splitext(image)[0] + ".txt"
    shutil.copy(os.path.join(labels_folder, label_file), os.path.join(root_val_labels_folder, label_file))


### Get the classes names

In [ ]:
class_list = []

with open('/content/ada_batch_1_augmented_yolov8/classes.txt', 'r') as file:
    for line in file:
        # Remove leading and trailing whitespaces and add to the list
        class_list.append(line.strip())

# Print the list
print(class_list)


['10_Relationship', '11_Company_or_plan', '12_Policyholder_details', '13_DOB', '14_Gender', '15_SSN', '16_Plan_Number', '17_Employer_Name', '18_Relationship', '19_Use', '1_Type_of_Transaction', '20_Name', '21_DOB', '22_Gender', '23_Patient_ID', '24_31_Table', '2_Pre_Number', '31_A_Other_Fee', '32_Total_Fee', '33_Missing_Teeth', '34_A_Diag_Codes', '34_Code_list_Qualifier', '35_Remarks', '36_Signature', '37_Signature', '38_Place_of_treatment', '39_Enclosures', '3_Company_address', '40_Orthodontics', '41_Date', '42_Months_remaining', '43_Prosthesis', '44_Date', '45_Treatment_resulting_form', '46_Date_of_accident', '47_Auto_Accident_state', '48_Dentist_Address', '49_NPI', '4_Other_Coverage', '50_Licence_Number', '51_SSN_TIN', '52_A_Addl_Provider_id', '52_Phone_Number', '53_Signature', '54_NPI', '55_Licence_Number', '56_A_Provider_Spl_Code', '56_Address', '57_Phone_Number', '58_Addl_provider_id', '5_Name', '6_DOB', '7_Gender', '8_SSN', '9_Plan_Number']


### Create `data.yaml`

In [ ]:
import yaml
from glob import glob
with open('/content/data/train.txt', 'w') as f:
    for path in glob('/content/data/images/train/*'):
        f.write(path+'\n')

with open('/content/data/val.txt', 'w') as f:
    for path in glob('/content/data/images/valid/*'):
        f.write(path+'\n')

data = dict(
    train = '/content/data/images/train',
    val = '/content/data/images/valid',

    nc    = len(class_list), # number of classes 2
    names = class_list
    )

with open('/content/data/data.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

In [ ]:
!pip install ultralytics

### Start training

In [ ]:
from ultralytics import YOLO

In [ ]:
import os
os.environ['WANDB_DISABLED'] = 'true'

In [ ]:
model = YOLO(model="yolov8n.pt")

100%|██████████| 6.23M/6.23M [00:00<00:00, 77.8MB/s]


In [ ]:
results = model.train(data='/content/data/data.yaml', epochs=1000, imgsz=1440, batch=8, project='/content/drive/MyDrive/001_projects/FSL/yolo_ada_initial_model_v1')

Ultralytics YOLOv8.0.238 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/data/data.yaml, epochs=1000, time=None, patience=50, batch=8, imgsz=1440, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/001_projects/FSL/yolo_ada_initial_model_v1, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=Fa

train: Scanning /content/data/labels/train.cache... 252 images, 0 backgrounds, 0 corrupt: 100%|██████████| 252/252 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/data/labels/valid.cache... 3 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3/3 [00:00<?, ?it/s]


Plotting labels to /content/drive/MyDrive/001_projects/FSL/yolo_ada_initial_model_v1/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000169, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
1000 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

   158/1000      10.4G     0.4174     0.3684     0.8208        529       1440: 100%|██████████| 32/32 [00:48<00:00,  1.50s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  6.96it/s]


                   all          3        160      0.959      0.994      0.995       0.92

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


   159/1000      11.7G     0.4151     0.3691     0.8208        377       1440: 100%|██████████| 32/32 [00:49<00:00,  1.53s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  4.87it/s]


                   all          3        160       0.94          1      0.995      0.916

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


   160/1000      9.81G     0.4353     0.3796     0.8247        723       1440:  44%|████▍     | 14/32 [00:18<00:21,  1.18s/it]

In [ ]:
from google.colab import runtime
runtime.unassign()

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import gc
gc.collect()

21013